### Read training, dev and unlabeled test data

The following provides a starting code (Python 3) of how to read the labeled training and dev sentence pairs, and unlabeled test sentence pairs, into lists.

In [1]:
import csv

In [ ]:
train, dev, test = [], [], []

In [2]:
with open('pnli_train.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[2] will be the label (0 or 1). x[0] and x[1] will be the sentence pairs.
        train.append(x)
print (len(train))
print (train[:3])

5983
[['Sometimes do exercise.', 'A person typically desire healthy life.', '1'], ['Who eats junk foods.', 'A person typically desire healthy life.', '0'], ['A person is sick.', 'A person typically desire healthy life.', '1']]


In [3]:
with open('pnli_dev.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[2] will be the label (0 or 1). x[0] and x[1] will be the sentence pairs.
        dev.append(x)
print (len(dev))
print (dev[:3])

1055
[['A person is looking for accuracy.', 'A person typically desires accurate results.', '1'], ['A person does not care for accuracy.', 'A person typically desires accurate results.', '0'], ['The person double checks their data.', 'A person typically desires accurate results.', '1']]


In [4]:
with open('pnli_test_unlabeled.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[0] and x[1] will be the sentence pairs.
        test.append(x)
print (len(test))
print (test[:3])

4850
[['The people want to have a romantic and pleasant feel.', 'People typically does desire to smell violets.'], ['The contract is to buy products from you.', 'Getting contract typically cause to make money or spend money.'], ['Train station is closed.', 'Line can typically be used to move train along tracks.']]


### Main Code Body

You may choose to experiment with different methods using your program. However, you need to embed the training and inference processes at here. We will use your prediction on the unlabeled test data to grade, while checking this part to understand how your method has produced the predictions.

In [5]:
import pandas as pd
train_data=pd.DataFrame(train,columns=["precondition","statement","labels"])

In [6]:
train_data.head()

,precondition,statement,labels
0,Sometimes do exercise.,A person typically desire healthy life.,1
1,Who eats junk foods.,A person typically desire healthy life.,0
2,A person is sick.,A person typically desire healthy life.,1
3,A person is dead.,A person typically desire healthy life.,0
4,A person eats properly and do exercise regularly.,A person typically desire healthy life.,1


In [7]:
train_data["labels"] = train_data["labels"].astype(str).astype(int)

In [8]:
train_data.dtypes

precondition    object
statement       object
labels           int64
dtype: object

In [9]:
dev_data=pd.DataFrame(dev,columns=["precondition","statement","labels"])

In [10]:
dev_data["labels"] = dev_data["labels"].astype(str).astype(int)

In [11]:
test_data=pd.DataFrame(test,columns=["precondition","statement"])
test_data.head()

,precondition,statement
0,The people want to have a romantic and pleasan...,People typically does desire to smell violets.
1,The contract is to buy products from you.,Getting contract typically cause to make money...
2,Train station is closed.,Line can typically be used to move train along...
3,There is no water for driving the boats.,People typically desires drive boats for fun.
4,The poet is busy.,Poet can typically be used for creating poetry.


In [12]:
!pip uninstall transformers

In [13]:
!pip install --no-cache-dir transformers sentencepiece

     |████████████████████████████████| 4.0 MB 7.1 MB/s 
     |████████████████████████████████| 1.2 MB 36.7 MB/s 
     |████████████████████████████████| 77 kB 37.6 MB/s 
     |████████████████████████████████| 596 kB 40.9 MB/s 
     |████████████████████████████████| 6.6 MB 19.5 MB/s 
     |████████████████████████████████| 895 kB 34.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [14]:
from transformers import BertTokenizer,TFBertModel,TFAutoModel,AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("roberta-base",use_fast=False)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [15]:
def encoding_precondition(data):
    tokens_list=list()
    tokens_list.append('[CLS]')
    tokens_list+=list(tokenizer.tokenize(data))
    return(tokenizer.convert_tokens_to_ids(tokens_list))
def encode_statement(data):
    tokens_list=list()
    tokens_list.append('[sep]')
    tokens_list+=list(tokenizer.tokenize(data))
    tokens_list.append('[sep]')
    return(tokenizer.convert_tokens_to_ids(tokens_list))
tokenized=[]
for i in range(len(train_data)):
    pre=encoding_precondition(train_data['precondition'][i])
    st=encode_statement(train_data['statement'][i])
    tokenized.append(pre+st)
train_data['tokenized']=tokenized
train_data.head()

,precondition,statement,labels,tokenized
0,Sometimes do exercise.,A person typically desire healthy life.,1,"[3, 13624, 109, 3325, 4, 3, 250, 621, 3700, 47..."
1,Who eats junk foods.,A person typically desire healthy life.,0,"[3, 12375, 24923, 15163, 6592, 4, 3, 250, 621,..."
2,A person is sick.,A person typically desire healthy life.,1,"[3, 250, 621, 16, 4736, 4, 3, 250, 621, 3700, ..."
3,A person is dead.,A person typically desire healthy life.,0,"[3, 250, 621, 16, 1462, 4, 3, 250, 621, 3700, ..."
4,A person eats properly and do exercise regularly.,A person typically desire healthy life.,1,"[3, 250, 621, 24923, 5083, 8, 109, 3325, 4595,..."


In [16]:
tokenized_dev=[]
for i in range(len(dev_data)):
    pre=encoding_precondition(dev_data['precondition'][i])
    st=encode_statement(dev_data['statement'][i])
    tokenized_dev.append(pre+st)
dev_data['tokenized']=tokenized_dev
dev_data.head()

,precondition,statement,labels,tokenized
0,A person is looking for accuracy.,A person typically desires accurate results.,1,"[3, 250, 621, 16, 546, 13, 8611, 4, 3, 250, 62..."
1,A person does not care for accuracy.,A person typically desires accurate results.,0,"[3, 250, 621, 473, 45, 575, 13, 8611, 4, 3, 25..."
2,The person double checks their data.,A person typically desires accurate results.,1,"[3, 133, 621, 1457, 6240, 49, 414, 4, 3, 250, ..."
3,The person speeds through the experiment.,A person typically desires accurate results.,0,"[3, 133, 621, 9706, 149, 5, 9280, 4, 3, 250, 6..."
4,A person is studying well.,A person typically desires accurate results.,1,"[3, 250, 621, 16, 7739, 157, 4, 3, 250, 621, 3..."


In [17]:
mask_data_train=[]
mask_data_dev=[]
for i in range(len(train_data)):
    padded_seq=tokenizer(train_data['precondition'][i],train_data['statement'][i],padding=True,add_special_tokens=True)
    mask_data_train.append(padded_seq)
for i in range(len(dev_data)):
    padded_seq_dev=tokenizer(dev_data['precondition'][i],dev_data['statement'][i],padding=True,add_special_tokens=True)
    mask_data_dev.append(padded_seq_dev)

In [19]:
df_train=train_data
df_train['masked']=mask_data_train
df_train.head(5)

,precondition,statement,labels,tokenized,masked
0,Sometimes do exercise.,A person typically desire healthy life.,1,"[3, 13624, 109, 3325, 4, 3, 250, 621, 3700, 47...","[input_ids, attention_mask]"
1,Who eats junk foods.,A person typically desire healthy life.,0,"[3, 12375, 24923, 15163, 6592, 4, 3, 250, 621,...","[input_ids, attention_mask]"
2,A person is sick.,A person typically desire healthy life.,1,"[3, 250, 621, 16, 4736, 4, 3, 250, 621, 3700, ...","[input_ids, attention_mask]"
3,A person is dead.,A person typically desire healthy life.,0,"[3, 250, 621, 16, 1462, 4, 3, 250, 621, 3700, ...","[input_ids, attention_mask]"
4,A person eats properly and do exercise regularly.,A person typically desire healthy life.,1,"[3, 250, 621, 24923, 5083, 8, 109, 3325, 4595,...","[input_ids, attention_mask]"


In [20]:
df_dev=dev_data
df_dev['masked']=mask_data_dev
df_dev.head(5)

,precondition,statement,labels,tokenized,masked
0,A person is looking for accuracy.,A person typically desires accurate results.,1,"[3, 250, 621, 16, 546, 13, 8611, 4, 3, 250, 62...","[input_ids, attention_mask]"
1,A person does not care for accuracy.,A person typically desires accurate results.,0,"[3, 250, 621, 473, 45, 575, 13, 8611, 4, 3, 25...","[input_ids, attention_mask]"
2,The person double checks their data.,A person typically desires accurate results.,1,"[3, 133, 621, 1457, 6240, 49, 414, 4, 3, 250, ...","[input_ids, attention_mask]"
3,The person speeds through the experiment.,A person typically desires accurate results.,0,"[3, 133, 621, 9706, 149, 5, 9280, 4, 3, 250, 6...","[input_ids, attention_mask]"
4,A person is studying well.,A person typically desires accurate results.,1,"[3, 250, 621, 16, 7739, 157, 4, 3, 250, 621, 3...","[input_ids, attention_mask]"


In [69]:
import tensorflow as tf
from tensorflow import keras

In [70]:
max_len=31
def build_model():
    encoder=TFAutoModel.from_pretrained('roberta-base')
    input_word_ids=tf.keras.Input(shape=(max_len,),dtype=tf.int32,name="input_word_ids")  
    input_mask=tf.keras.Input(shape=(max_len,),dtype=tf.int32,name="input_mask")
    embedding=encoder([input_word_ids,input_mask])[0] 
    output=tf.keras.layers.Dense(2,activation='softmax')(embedding[:,0,:])
    model=tf.keras.Model(inputs=[input_word_ids,input_mask],outputs=output) 
    loss_fn=keras.losses.SparseCategoricalCrossentropy() 
    model.compile(tf.keras.optimizers.Adam(lr=1e-5),loss=loss_fn,metrics=['accuracy'])
    return(model)

In [71]:
def input_data_con(data):
    inputs={'input_word_ids':list(),'input_mask':list()}
    for i in data:
        inputs['input_word_ids'].append(i['input_ids'])
        inputs['input_mask'].append(i['attention_mask'])
    inputs['input_word_ids']=tf.ragged.constant(inputs['input_word_ids']).to_tensor()
    inputs['input_mask']=tf.ragged.constant(inputs['input_mask']).to_tensor()
    return(inputs)

In [66]:
df_train_input=input_data_con(df_train['masked'].values)
for key in df_train_input.keys():
    df_train_input[key] =df_train_input[key][:,:max_len]

In [72]:
df_dev_input=input_data_con(df_dev['masked'].values)
for key in df_dev_input.keys():
    df_dev_input[key]=df_dev_input[key][:,:max_len]

In [73]:
model=build_model()
model.summary()

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 31)]         0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 31)]         0           []                               
                                                                                                  
 tf_roberta_model_8 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_word_ids[0][0]',         
 odel)                          thPoolingAndCrossAt               'input_mask[0][0]']             
                                tentions(last_hidde                                               
                                n_state=(None, 31,                                          

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [74]:
model.fit(df_train_input,df_train['labels'],verbose=2,validation_data=(df_dev_input,df_dev['labels']),epochs=8) 

Epoch 1/8
187/187 - 157s - loss: 0.5715 - accuracy: 0.6784 - val_loss: 0.4086 - val_accuracy: 0.8246 - 157s/epoch - 841ms/step
Epoch 2/8
187/187 - 96s - loss: 0.3609 - accuracy: 0.8521 - val_loss: 0.3438 - val_accuracy: 0.8682 - 96s/epoch - 513ms/step
Epoch 3/8
187/187 - 94s - loss: 0.2865 - accuracy: 0.8882 - val_loss: 0.3226 - val_accuracy: 0.8796 - 94s/epoch - 502ms/step
Epoch 4/8
187/187 - 92s - loss: 0.2335 - accuracy: 0.9116 - val_loss: 0.3428 - val_accuracy: 0.8806 - 92s/epoch - 493ms/step
Epoch 5/8
187/187 - 93s - loss: 0.1846 - accuracy: 0.9275 - val_loss: 0.3155 - val_accuracy: 0.8853 - 93s/epoch - 499ms/step
Epoch 6/8
187/187 - 97s - loss: 0.1428 - accuracy: 0.9445 - val_loss: 0.3207 - val_accuracy: 0.8863 - 97s/epoch - 520ms/step
Epoch 7/8
187/187 - 93s - loss: 0.1102 - accuracy: 0.9594 - val_loss: 0.4472 - val_accuracy: 0.8787 - 93s/epoch - 497ms/step
Epoch 8/8
187/187 - 95s - loss: 0.0929 - accuracy: 0.9639 - val_loss: 0.4788 - val_accuracy: 0.8806 - 95s/epoch - 509ms/ste

In [75]:
tokenized_test=[]
for i in range(len(test_data)):
    pre=encoding_precondition(test_data['precondition'][i])
    st=encode_statement(test_data['statement'][i])
    tokenized_test.append(pre+st)
test_data['tokenized']=tokenized_test
test_data.head()

,precondition,statement,tokenized
0,The people want to have a romantic and pleasan...,People typically does desire to smell violets.,"[3, 133, 82, 236, 7, 33, 10, 8728, 8, 16219, 6..."
1,The contract is to buy products from you.,Getting contract typically cause to make money...,"[3, 133, 1355, 16, 7, 907, 785, 31, 47, 4, 3, ..."
2,Train station is closed.,Line can typically be used to move train along...,"[3, 40249, 1992, 16, 1367, 4, 3, 18997, 64, 37..."
3,There is no water for driving the boats.,People typically desires drive boats for fun.,"[3, 970, 16, 117, 514, 13, 1428, 5, 8934, 4, 3..."
4,The poet is busy.,Poet can typically be used for creating poetry.,"[3, 133, 16893, 16, 3610, 4, 3, 26170, 594, 64..."


In [76]:
mask_test=[]
for i in range(len(test_data)):
    padded_seq=tokenizer(test_data['precondition'][i],test_data['statement'][i],padding=True,add_special_tokens=True)
    mask_test.append(padded_seq)
test_data['masked'] =mask_test
df_test_input=input_data_con(test_data['masked'].values)
for key in df_test_input.keys():
    df_test_input[key]=df_test_input[key][:,:max_len]

### Output Prediction Result File

You will need to submit a prediction result file. It should have 2028 lines, every line should be either 0 or 1, which is your model's prediction on the respective test set instance.

In [77]:
results = []

In [78]:
import numpy as np

In [79]:
results=[np.argmax(i) for i in model.predict(df_test_input)] 

In [80]:
# suppose you had your model's predictions on the 2028 test cases read from test_enc_unlabeled.tsv, and 
#those results are in the list called 'results'
assert (len(results) == 4850)

In [81]:
# make sure the results are not float numbers, but intergers 0 and 1
results = [int(x) for x in results]

In [82]:
# write your prediction results to 'upload_predictions.txt' and upload that later
with open('upload_predictions.txt', 'w', encoding = 'utf-8') as fp:
    for x in results:
        fp.write(str(x) + '\n')